# Efficient Yelp API Calls (Core)
- Zach Hanson

## Libraries

In [22]:
#Standard libraries
import pandas as pd
import numpy as np

#Yelp API
from yelpapi import YelpAPI

#Additional imports
import os
import json
import math
import time
from tqdm.notebook import tqdm_notebook

## Functions

In [20]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

## Loading Credentials

In [3]:
#Loading Credentials
with open('/Users/Zach/.secret/yelp_api.json', 'r') as f:
    login = json.load(f)

In [5]:
#Instantiate YelpAPI
yelp = YelpAPI(login['api-key'], timeout_s = 5)

## Defining Search Terms

In [6]:
#Set call parameters
location = 'Portland, OR 97229'
term = 'barbeque'

In [7]:
#Folder for saving data

FOLDER = 'Data/'
os.makedirs(FOLDER, exist_ok = True)

In [9]:
#JSON filename
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

In [10]:
JSON_FILE

'Data/Portland-barbeque.json'

## Creating JSON

In [24]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp.search_query(location=location,
                                term=term,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/Portland-barbeque.json already exists. Deleting previous file...
[i] Data/Portland-barbeque.json not found. Saving empty list to new file.
- 0 previous results found.


70

## Getting Results

In [26]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp.search_query(location=location,
                                    term=term, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/70 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


## Converting Results to Dataframe


In [27]:
df = pd.read_json(JSON_FILE)
display(df.head(), df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,VnZAD0Sj-QR7I_iGNsYuig,the-meating-place-cafe-hillsboro-3,The Meating Place Cafe,https://s3-media3.fl.yelpcdn.com/bphoto/zInTti...,False,https://www.yelp.com/biz/the-meating-place-caf...,426,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.5,"{'latitude': 45.56707705675265, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '6495 NW Cornelius Pass Rd', 'add...",+15035330624,(503) 533-0624,6012.121223
1,HkVKHDwcKOSFWogsV0CJMg,bottos-bbq-portland-3,Botto's BBQ,https://s3-media3.fl.yelpcdn.com/bphoto/XzaYy7...,False,https://www.yelp.com/biz/bottos-bbq-portland-3...,137,"[{'alias': 'bbq', 'title': 'Barbeque'}]",5.0,"{'latitude': 45.500417, 'longitude': -122.653972}",[delivery],$$,"{'address1': '3120 SE Milwaukie Ave', 'address...",+15032066936,(503) 206-6936,14163.559332
2,HpJH0W2zq8f9nGh0394uMA,wolfs-head-smokehouse-beaverton-2,Wolf's Head Smokehouse,https://s3-media2.fl.yelpcdn.com/bphoto/AHYA53...,False,https://www.yelp.com/biz/wolfs-head-smokehouse...,71,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.0,"{'latitude': 45.489246, 'longitude': -122.808394}","[delivery, pickup]",$$,"{'address1': '4250 SW Rose Biggi Ave', 'addres...",+15039974298,(503) 997-4298,7138.188341
3,9k6kYNc6f7-G4pFLOZPbrw,smoke-squad-bbq-tigard-2,Smoke Squad BBQ,https://s3-media2.fl.yelpcdn.com/bphoto/flPS43...,False,https://www.yelp.com/biz/smoke-squad-bbq-tigar...,123,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.5,"{'latitude': 45.4404395, 'longitude': -122.749...","[delivery, pickup]",$$,"{'address1': '11540 SW Pacific Hwy', 'address2...",+15033800688,(503) 380-0688,13660.748062
4,OTaBVE8hN_sHf1shpR_Flg,podnahs-pit-barbecue-portland-3,Podnah's Pit Barbecue,https://s3-media1.fl.yelpcdn.com/bphoto/aAlm-w...,False,https://www.yelp.com/biz/podnahs-pit-barbecue-...,1547,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.0,"{'latitude': 45.56286, 'longitude': -122.64837}","[delivery, pickup]",$$,"{'address1': '1625 NE Killingsworth St', 'addr...",+15032813700,(503) 281-3700,13375.030161


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,xArHQSOw-yYa2WKmJGpP2Q,dixie-tavern-portland-2,Dixie Tavern,https://s3-media2.fl.yelpcdn.com/bphoto/z0ApTe...,False,https://www.yelp.com/biz/dixie-tavern-portland...,145,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",3.0,"{'latitude': 45.52369, 'longitude': -122.67311}","[pickup, delivery]",$$,"{'address1': '32 NW 3rd Ave', 'address2': '', ...",+15032349431,(503) 234-9431,11864.206925
996,K2k3pcyxJg2iYLZAaugtAA,taqueria-portland-portland-3,Taqueria Portland,https://s3-media1.fl.yelpcdn.com/bphoto/5BXLJF...,False,https://www.yelp.com/biz/taqueria-portland-por...,207,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 45.5168299, 'longitude': -122.657...","[pickup, delivery]",$,"{'address1': '820 SE 8th Ave', 'address2': '',...",+15032327000,(503) 232-7000,13240.339972
997,YgSpVN4zStgkbULQTKtBZA,grand-central-bowl-and-arcade-portland,Grand Central Bowl & Arcade,https://s3-media4.fl.yelpcdn.com/bphoto/-jDjB-...,False,https://www.yelp.com/biz/grand-central-bowl-an...,31,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",3.5,"{'latitude': 45.51706568056381, 'longitude': -...",[delivery],NaN,"{'address1': '808 SE Morrison St', 'address2':...",+15032362695,(503) 236-2695,13251.281810
998,4ilqVZA0ifTQtzTcUHMWug,tara-thai-northwest-portland,Tara Thai Northwest,https://s3-media2.fl.yelpcdn.com/bphoto/9iu3OL...,False,https://www.yelp.com/biz/tara-thai-northwest-p...,191,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",3.5,"{'latitude': 45.53224, 'longitude': -122.69834}","[pickup, delivery]",$$,"{'address1': '1310 NW 23rd Ave', 'address2': '...",+15032227840,(503) 222-7840,9713.447334
999,K1jNnR8wWgHEWMRTGmKAaw,tuk-tuk-portland-2,Tuk Tuk,https://s3-media3.fl.yelpcdn.com/bphoto/1KtS5Y...,False,https://www.yelp.com/biz/tuk-tuk-portland-2?ad...,124,"[{'alias': 'thai', 'title': 'Thai'}]",3.5,"{'latitude': 45.5769462585449, 'longitude': -1...","[pickup, delivery]",$$,"{'address1': '3226 N Lombard St', 'address2': ...",+15037197796,(503) 719-7796,9680.143157
